In [1]:
# import jupyter notebook version of dash framework
from jupyter_dash import JupyterDash as Dash
# import callbacks
from dash import Input, Output
# import html elements for dash
from dash import html
# Import markdown html translations
from dash import dcc
# Import warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')
# import os
import os

# Define image dataset path to use for app
image_dataset_path = "../data/testing"

# Define the subdirectories for the lesion types
subdirectories = ["no_tumor", "glioma", "meningioma", "pituitary"]

# Function to get image file names from a directory
def get_image_files(directory):
    files = os.listdir(directory)
    return [f for f in files if f.endswith('.jpg')]

# Create dropdown options from subdirectories
dropdown_options = [
    {"label": subdir.capitalize(), "value": subdir} 
    for subdir in subdirectories
]

# Create the dash app
app = Dash(__name__)

# Create markdown for our dashboard
markdown_1 = dcc.Markdown("""
    # Brain Tumor Lesion Assesment
""")

markdown_2 = dcc.Markdown("""
    The model integrated in into this dashbaord has the capability to predict different brian lesions.
    The four supported brain lession classification are Meningioma, Pituitary, Glioma tumors. 
    The fourth possible prediction would be a No Tumor classification.
""")

# Create the dropdowns to choose Lesion type and images
dropdowns = []
for lesion_type in subdirectories:
    image_files = get_image_files(os.path.join(image_dataset_path, lesion_type))
    options = [{"label": file, "value": file} for file in image_files]
    dropdown = html.Div([
        f"Select an Image for {lesion_type.capitalize()}:",
        dcc.Dropdown(
            id=f"image-dropdown-{lesion_type}",
            options=options
        )
    ])
    dropdowns.append(dropdown)

# Create empty div to hold selected image
selected_image_div = html.Div(id='selected-image-upload')

# Set the layout
app.layout = html.Div(children=[
    html.Div(markdown_1),
    html.Div(markdown_2),
    *dropdowns,
    html.Div(selected_image_div),
])

# Create empty html.Img components for each dropdown to display selected images
selected_image_divs = [html.Img(id=f'selected-image-div-{lesion_type}', style={'width': '300px'}) for lesion_type in subdirectories]

# Callbacks to update selected image based on dropdown selection
for lesion_type in subdirectories:
    @app.callback(
        Output(f'selected-image-div-{lesion_type}', 'src'),
        [Input(f'image-dropdown-{lesion_type}', 'value')]
    )
    def update_selected_image(selected_image_path):
        if selected_image_path is not None:
            image_src = f'{image_dataset_path}/{lesion_type}/{selected_image_path}'
            print("Selected Image Path:", image_src)  # Print selected image path for debugging
            return image_src
        else:
            return ""

# # Callback to update selected image based on dropdown selection
# @app.callback(
#     Output('selected-image-upload', 'children'),
#     [Input(f'image-dropdown-{lesion_type}', 'value') for lesion_type in subdirectories]
# )
# # Update the image selection
# def update_selected_image(*selected_image_paths):
#     images = []
#     for lesion_type, path in zip(subdirectories, selected_image_paths):
#         if path is not None:
#             image_src = f'{image_dataset_path}/{lesion_type}/{path}'
#             print("Image Source:", image_src)  # Print image source for debugging
#             images.append(html.Img(src=image_src, style={'width': '300px'}))
#     return [images]

In [2]:
app.run_server(mode="inline", host="localhost", port=5000)